In [ ]:
####
# Rohit Musti
# rmusti@americanforests.org
####

from tqdm import tqdm
import pandas as pd
import rasterio as rio
import rasterstats as rs
import geopandas as gpd
import pandas as pd
import os, shutil
import numpy as np
import requests
from bs4 import BeautifulSoup
from glob import glob

In [ ]:

print("setting file paths")
home_dir = "Y:/CommunityReLeaf/TreeEquityScore/"
census_block_groups_dir = home_dir + "CensusBlockGroups/"
ards_path = home_dir + "landsat/ards/"
wrs_path = home_dir + "landsat/wrs2/"

# Determine What Data to Download

In [ ]:
ards = gpd.GeoDataFrame.from_file(ards_path)
wrs = gpd.GeoDataFrame.from_file(wrs_path)

ards = ards.to_crs(wrs.crs)
res = gpd.sjoin(ards, wrs)

In [ ]:
# pull data for amazon's s3 landsat bucket
print("pulling scenes from amazon")
s3_scenes = pd.read_csv('http://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz', compression='gzip')

In [ ]:
def convertFromARDStoWRS2(h,v):
    '''
    author: rmusti
    args: h (the h from ards file), v (the v from ards file)
    returns: a list of (path, row) coordinates that intersect with the ards block
    '''
    res_temp = res[res.h==h]
    res_temp = res_temp[res_temp.v==v]
    return list(pd.DataFrame(res_temp.groupby(['PATH', 'ROW'])).iloc[:,0])

In [ ]:
pras = {}
for i in tqdm(os.listdir(f"{home_dir}clipped_block_groups")):
    if i=="ak" or i =="hi" or i != "il": continue
    filename = f"{home_dir}clipped_block_groups/{i}/{i}_ed.shp"
    focus_area = gpd.read_file(filename)
    ards = ards.to_crs(focus_area.crs)
    ards_intersection = gpd.sjoin(focus_area, ards, how='left', op='intersects')
    unique_hvs = list(pd.DataFrame(ards_intersection.groupby(["h", "v"])).iloc[:,0])
    unique_path_rows = []
    for h,v in unique_hvs:
        unique_path_rows.extend(convertFromARDStoWRS2(h, v))
    unique_path_rows = list(set(unique_path_rows))
    pras[i] = unique_path_rows

In [ ]:
start_month = 6
end_month = 10
cloud_cover_threshold = 25
state_scenes = {}
num_scenes = 5
for i in pras.keys():
    state_list = []
    for path, row in (pras[i]):
        scenes = s3_scenes[(s3_scenes.path == path) & (s3_scenes.row == row) & 
                           (s3_scenes.cloudCover <= cloud_cover_threshold) & 
                           (~s3_scenes.productId.str.contains('_T2')) & (~s3_scenes.productId.str.contains('_RT'))]
        scenes['acquisitionMonth'] = scenes.acquisitionDate.str.split("-").str[1].astype(int) # extract month
        scenes = scenes[(scenes.acquisitionMonth >= start_month) & (scenes.acquisitionMonth <= end_month)] # only scenes in month range
        scenes = scenes.sort_values("cloudCover", ascending=True).iloc[:num_scenes]
        state_list.append(scenes)
        if len(scenes) == 0: print(i, path, row, len(scenes)>0)
    state_list = pd.concat(state_list)
    state_scenes[i] = state_list

# Download Data

In [ ]:
for i in tqdm(state_scenes):
    if i not in os.listdir(home_dir+"landsat"):
        folder = home_dir+f"landsat/{i}"
        os.mkdir(folder, 0o666)
    for _, row in tqdm(state_scenes[i].iterrows()):
        if row.productId in os.listdir(f"{home_dir}/landsat/{i}/"): continue
        response = requests.get(row.download_url)

        # If the response status code is fine (200)
        if response.status_code == 200:
            # Import the html to beautiful soup
            html = BeautifulSoup(response.content, 'html.parser')
            # Create the dir where we will put this image files.
            entity_dir = os.path.join(f"{home_dir}/landsat/{i}/", row.productId)
            os.makedirs(entity_dir, exist_ok=True)

            # Second loop: for each band of this image that we find using the html <li> tag
            for li in html.find_all('li'):
                    # Get the href tag
                current_file = li.find_next('a').get('href')
                if "B10.TIF" in current_file or "B11.TIF" in current_file or "MTL" in current_file:
                    response = requests.get(row.download_url.replace('index.html', current_file), stream=True)
                    with open(os.path.join(entity_dir, current_file), 'wb') as output:
                        shutil.copyfileobj(response.raw, output)
                    del response
        else:
            print('error:',response)
    

# Process Downloaded Data

In [ ]:
#temperature calculation for a raster file

def temp_calc(constants_dict, band10, band11, kelvin_constant=273.15):
    band10_radiance = float(constants_dict['RADIANCE_MULT_BAND_10'])*band10 + float(constants_dict['RADIANCE_ADD_BAND_10'])
    band11_radiance = float(constants_dict['RADIANCE_MULT_BAND_11'])*band11 + float(constants_dict['RADIANCE_ADD_BAND_11'])

    # TODO: convert this to a lambda function
    band10_sattemp = (float(constants_dict['K2_CONSTANT_BAND_10']) / np.log((float(   constants_dict['K1_CONSTANT_BAND_10'])/band10_radiance) + 1)) - kelvin_constant 
    band11_sattemp = (float(constants_dict['K2_CONSTANT_BAND_11']) / np.log((float(constants_dict['K1_CONSTANT_BAND_11'])/band11_radiance) + 1)) - kelvin_constant 

    # TODO: convert this to a lambda function
    band10_sattemp_far = (band10_sattemp * 1.8) + 32
    band11_sattemp_far = (band11_sattemp * 1.8) + 32

    sattemp_avg = (band10_sattemp + band11_sattemp) / 2.0
    sattemp_avg_far = (band10_sattemp_far + band11_sattemp_far) / 2.0
    return sattemp_avg_far

In [ ]:
def get_bands(landsat_raw, current_band):
    current_bands_path = os.path.join(landsat_raw, current_band)
    current_bands = [i for i in os.listdir(current_bands_path) if 'ovr' not in i]
    bands10 = [i for i in current_bands if 'B10' in i]
    bands11 = [i for i in current_bands if 'B11' in i]
    bands_meta = [i for i in current_bands if 'MTL' in i]
    assert(len(bands10)== len(bands11) == len(bands_meta)==1)
    band10_path = os.path.join(current_bands_path, bands10[0])
    band11_path = os.path.join(current_bands_path, bands11[0])
    bands_meta_path = os.path.join(current_bands_path, bands_meta[0])
    return (band10_path, band11_path, bands_meta_path)

In [ ]:
for i in tqdm(os.listdir(home_dir+"landsat")):
    if len(i) != 2: continue
    print("processing", i)
    for q in (os.listdir(home_dir+f"landsat/{i}")):
        if 'temperature.tif' in os.listdir(f'{home_dir}landsat/{i}/{q}'): continue
        band10_path, band11_path, bands_meta_path = get_bands(f"{home_dir}landsat/{i}", q)
        with rio.open(band10_path) as band10_in, rio.open(band11_path) as band11_in, open(bands_meta_path, 'r') as band_meta_in:
            target_crs = band10_in.profile['crs']
            bands_meta = band10_in
            band10 = band10_in.read()
            band11 = band11_in.read()
            band_meta = [i.split() for i in band_meta_in.readlines()]
            band10 = np.ma.masked_array(band10, mask=(band10 == 0)).astype(float)
            band11 = np.ma.masked_array(band11, mask=(band11 == 0)).astype(float)
            constants_dict = {i[0]:i[-1] for i in band_meta}
            temperature_frame = temp_calc(constants_dict,band10,band11)
            temperature_frame[temperature_frame > 110] = -999 # set to no data
            temperature_frame[temperature_frame < 40] = -999 # set to no data
            with rio.Env():
                profile = band10_in.profile
                profile.update(
                    dtype=rio.uint8,
                    count=1,
                    compress='lzw')
                with rio.open(f'{home_dir}landsat/{i}/{q}/temperature.tif', 'w', **profile) as dst:
                    dst.write(temperature_frame.astype(rio.uint8))   

In [ ]:
for i in ([q for q in os.listdir(home_dir+"landsat") if len(q)<=2]):
    state = gpd.read_file(f"{home_dir}clipped_block_groups/{i}/{i}_ed.shp")
    counter = 0
    print(f'processing {i}')
    for frame in tqdm([z for z in os.listdir(f"{home_dir}landsat/{i}/groups/") if '.tif' in z and 'tif.' not in z]):
        temperature_path = f"{home_dir}landsat/{i}/groups/{frame}"
        with rio.open(temperature_path) as temperature_in:
            target_crs = temperature_in.profile['crs']
            state = state.to_crs(target_crs)
            temperature_meta = temperature_in
            temperature = temperature_in.read()
            temperature[temperature==25] = 0
            state = rs.zonal_stats(state, temperature[0],
                                                  nodata=0,
                                                  affine=temperature_in.profile['transform'],
                                                  geojson_out=True, copy_properties=True,
                                                  stats="mean")
            state = gpd.GeoDataFrame.from_features(state, crs=target_crs)
            state = state.rename(columns={'mean':f'temp_{frame}'})
            counter += 1
    state['avg_temp'] = state[[i for i in state.columns if 'temp' in i]].mean(axis=1)
    state.to_csv(f"{home_dir}clipped_block_groups_temperature/{i}_mean.csv")